<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.6: Generators: Types
**Prev: [Object Oriented Programming](3.5_object_oriented_programming.ipynb)**<br>
**Next: [Introduction to FIRRTL](4.1_firrtl_ast.ipynb)**

## Motivation
Scala 是一种强类型编程语言。这是一把双刃剑；一方面，许多在 Python（动态类型语言）中可以编译和执行的程序在 Scala 中会在编译时失败。另一方面，在 Scala 中编译的程序比类似的 Python 程序包含的运行时错误要少得多。

在本节中，我们的目标是让你熟悉类型作为 Scala 中的一级公民。虽然最初你可能会觉得生产力有限，但你很快就会学会理解编译时错误消息，并如何考虑类型系统来构建你的程序，以便为你捕捉更多的错误。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# Static Types<a name="types-in-scala"></a>

## Types in Scala

Scala 中的所有对象（objects）都有一个类型，通常是对象的类。
让我们看一些例子：

In [ ]:
println(10.getClass)
println(10.0.getClass)
println("ten".getClass)

When you declare your own class, it has an associated type.

In [ ]:
class MyClass {
    def myMethod = ???
}
println(new MyClass().getClass)

虽然不是必需的，但强烈建议**为所有函数声明定义输入和输出类型**。这将使 Scala 编译器能够捕捉函数的不正确使用。

In [ ]:
def double(s: String): String = s + s
// Uncomment the code below to test it
// double("hi")      // Proper use of double
// double(10)        // Bad input argument!
// double("hi") / 10 // Inproper use of double's output!

Functions that don't return anything return type `Unit`.

In [ ]:
var counter = 0
def increment(): Unit = {
    counter += 1
}
increment()

## Scala vs. Chisel Types<a name="scala-vs-chisel-types"></a>

回顾：模块 2.2 讨论了 Chisel 类型和 Scala 类型之间的区别，例如

```scala
val a = Wire(UInt(4.W))
a := 0.U
```

是合法的，因为 `0.U` 的类型是 `UInt`（一种 Chisel 类型），而

```scala
val a = Wire(UInt(4.W))
a := 0
```

是不合法的，因为 0 的类型是 `Int`（一种 Scala 类型）。（译者注：UInt(4.W)是类型节点，0.U是硬件节点。这在模块2.4节有简要叙述。）

这同样适用于 `Bool`，它是一种不同于 `Boolean` 的 Chisel 类型。

```scala
val bool = Wire(Bool())
val boolean: Boolean = false
// 合法
when (bool) { ... }
if (boolean) { ... }
// 非法
if (bool) { ... }
when (boolean) { ... }
```

如果你犯了一个错误，将 `UInt` 和 `Int` 或 `Bool` 和 `Boolean` 混淆，Scala 编译器通常会为你捕捉到这个错误。
这是因为 Scala 的静态类型。
在编译时，编译器能够区分 Chisel 和 Scala 类型，并且能够理解 `if ()` 需要 `Boolean`，而 `when ()` 需要 `Bool`。

## Scala Type Coercion<a name="type-coercion"></a>

<!-- typeOf. Scala has a function called `typeOf[T]` which returns a type object for `T`. -->
<!-- This doesn't actually seem useful to Chisel users... -->

### asInstanceOf

`x.asInstanceOf[T]` 将对象 `x` 强制转换为类型 `T`。如果给定对象不能强制转换为类型 `T`，它会抛出异常。

In [ ]:
val x: UInt = 3.U
try {
  println(x.asInstanceOf[Int])
} catch {
  case e: java.lang.ClassCastException => println("As expected, we can't cast UInt to Int")
}

// But we can cast UInt to Data since UInt inherits from Data.
println(x.asInstanceOf[Data])

### Type Casting in Chisel

如果尝试运行下面的代码而不删除注释，将会出现错误。问题是什么？
它试图将一个 `UInt` 分配给一个 `SInt`，这是非法的。

Chisel 提供了一组类型转换函数。
最通用的是 `asTypeOf()`，如下所示。
一些 Chisel 对象还定义了 `asUInt()` 和 `asSInt()` 以及其他一些类型转换函数。

如果删除下面代码块中的 `//`，示例应该可以正常运行。

```scala
val a = Wire(UInt(4.W))
val b = Wire(SInt(4.W))
// b := a  // 这行会导致错误

b := a.asTypeOf(SInt(4.W))  // 正确的类型转换
```

In [ ]:
class TypeConvertDemo extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(4.W))
        val out = Output(SInt(4.W))
    })
    io.out := io.in//.asTypeOf(io.out)
}

test(new TypeConvertDemo) { c =>
      c.io.in.poke(3.U)
      c.io.out.expect(3.S)
      c.io.in.poke(15.U)
      c.io.out.expect(-1.S)
}

---
# Type Matching<a name="type-matching"></a>

## Match Operator
回顾一下，在 3.1 中介绍了 match 操作符。
类型匹配在编写类型泛型生成器时尤其有用。
下面的示例展示了一个可以添加类型为 `UInt` 或 `SInt` 的两个字面量的“生成器”。
后面的部分将更多地讨论如何编写类型泛型生成器。

**注意：在 Scala 中有更好且更安全的方法来编写类型泛型生成器**。

In [ ]:
class ConstantSum(in1: Data, in2: Data) extends Module {
    val io = IO(new Bundle {
        val out = Output(chiselTypeOf(in1)) // in case in1 is literal then just get its type
    })
    (in1, in2) match {
        case (x: UInt, y: UInt) => io.out := x + y
        case (x: SInt, y: SInt) => io.out := x + y
        case _ => throw new Exception("I give up!")
    }
}
println(getVerilog(dut = new ConstantSum(3.U, 4.U)))
println(getVerilog(dut = new ConstantSum(-3.S, 4.S)))
println(getVerilog(dut = new ConstantSum(3.U, 4.S)))


需要记住的是，Chisel 类型通常不应进行值匹配。
Scala 的 match 在电路详细生成（elaboration）期间执行，但你可能需要的是详细生成后（post-elaboration）的比较。
以下代码会产生语法错误：

In [ ]:
class InputIsZero extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(16.W))
        val out = Output(Bool())
    })
    io.out := (io.in match {
        // note that case 0.U is an error
        case (0.U) => true.B
        case _   => false.B
    })
}
println(getVerilog(new InputIsZero))

## Unapply
在执行 match 时，实际发生了什么？
Scala 是如何让你用 case classes 进行花哨的值匹配的，如下所示：
```scala
case class Something(a: String, b: Int)
val a = Something("A", 3)
a match {
    case Something("A", value) => value
    case Something(str, 3)     => 0
}
```

事实证明，为每个 case class 创建的伴生对象除了包含 **apply** 方法外，还包含 **unapply** 方法。
什么是 **unapply** 方法？

Scala 的 unapply 方法是另一种语法糖，使得 match 语句不仅能够匹配类型，还能够在匹配过程中**提取值**。

让我们看下面的例子。
出于某种原因，假设如果生成器被流水化，延迟为 `3*totalWidth`，否则延迟为 `2*someOtherWidth`。
由于 case classes 定义了 **unapply**，我们可以匹配 case class 中的值，如下所示：

```scala
case class Generator(pipelined: Boolean, totalWidth: Int, someOtherWidth: Int)

def calculateDelay(gen: Generator): Int = {
  gen match {
    case Generator(true, totalWidth, _) => 3 * totalWidth
    case Generator(false, _, someOtherWidth) => 2 * someOtherWidth
  }
}

val gen1 = Generator(true, 10, 5)
val gen2 = Generator(false, 10, 5)

println(calculateDelay(gen1))  // 输出 30
println(calculateDelay(gen2))  // 输出 10
```

在这个例子中，我们使用 unapply 提取 Generator case class 中的值，并根据 `pipelined` 字段的值计算延迟。

In [ ]:
case class SomeGeneratorParameters(
    someWidth: Int,
    someOtherWidth: Int = 10,
    pipelineMe: Boolean = false
) {
    require(someWidth >= 0)
    require(someOtherWidth >= 0)
    val totalWidth = someWidth + someOtherWidth
}

def delay(p: SomeGeneratorParameters): Int = p match {
    case SomeGeneratorParameters(_, sw, false) => sw * 2
    case sg @SomeGeneratorParameters(_, _, true) => sg.totalWidth * 3
}

println(delay(SomeGeneratorParameters(10, 10)))
println(delay(SomeGeneratorParameters(10, 10, true)))

如果你查看 `delay` 函数，你会注意到，除了匹配每个字符的类型，我们还在做以下几件事：
- 直接引用参数的内部值
- 有时直接匹配参数的内部值

这些都是因为编译器实现了 `unapply` 方法。请注意，unapply 只是语法糖；例如，以下两个 case 示例是等价的：
```scala
case p: SomeGeneratorParameters => p.sw * 2
case SomeGeneratorParameters(_, sw, _) => sw * 2
```

此外，还有更多的匹配语法和风格。以下两个 case 也是等价的，但第二个允许在引用父值的同时匹配内部值：
```scala
case SomeGeneratorParameters(_, sw, true) => sw
case sg @SomeGeneratorParameters(_, sw, true) => sw
```

最后，你可以将条件检查直接嵌入到 match 语句中，如以下等价示例的第三个所示：
```scala
case SomeGeneratorParameters(_, sw, false) => sw * 2
case s @SomeGeneratorParameters(_, sw, false) => s.sw * 2
case s: SomeGeneratorParameters if s.pipelineMe => s.sw * 2
```

所有这些语法都是通过类的伴生对象中的 Scala unapply 方法启用的。如果你想对一个类进行 unapply，但不想将其设为 case class，你可以手动实现 unapply 方法。以下示例演示了如何手动实现类的 apply 和 unapply 方法：

In [ ]:
class Boat(val name: String, val length: Int)
object Boat {
    def unapply(b: Boat): Option[(String, Int)] = Some((b.name, b.length))
    def apply(name: String, length: Int): Boat = new Boat(name, length)
}

def getSmallBoats(seq: Seq[Boat]): Seq[Boat] = seq.filter { b =>
    b match {
        case Boat(_, length) if length < 60 => true
        case Boat(_, _) => false
    }
}

val boats = Seq(Boat("Santa Maria", 62), Boat("Pinta", 56), Boat("Nina", 50))
println(getSmallBoats(boats).map(_.name).mkString(" and ") + " are small boats!")

## Partial Functions
这是一个简要概述；[本指南](https://twitter.github.io/scala_school/pattern-matching-and-functional-composition.html#PartialFunction) 有更详细的介绍。

Partial Functions 是只在其输入子集上定义的函数。
类似于 Option，Partial Function 可能对特定输入没有值。
这可以通过 `isDefinedAt(...)` 来测试。

Partial Functions 可以通过 `orElse` 链接在一起。

请注意，调用 Partial Function 时如果输入未定义，将导致运行时错误。例如，如果 Partial Function 的输入是用户定义的，则可能会发生这种情况。为了更类型安全，建议编写返回 Option 的函数。

In [ ]:
// Helper function to make this cell a bit less tedious.
def printAndAssert(cmd: String, result: Boolean, expected: Boolean): Unit = {
  println(s"$cmd = $result")
  assert(result == expected)
}

// Defined for -1, 2, 5, etc.
val partialFunc1: PartialFunction[Int, String] = {
  case i if (i + 1) % 3 == 0 => "Something"
}
printAndAssert("partialFunc1.isDefinedAt(2)", partialFunc1.isDefinedAt(2), true)
printAndAssert("partialFunc1.isDefinedAt(5)", partialFunc1.isDefinedAt(5), true)
printAndAssert("partialFunc1.isDefinedAt(1)", partialFunc1.isDefinedAt(1), false)
printAndAssert("partialFunc1.isDefinedAt(0)", partialFunc1.isDefinedAt(0), false)
println(s"partialFunc1(2) = ${partialFunc1(2)}")
try {
  println(partialFunc1(0))
} catch {
  case e: scala.MatchError => println("partialFunc1(0) = can't apply PartialFunctions where they are not defined")
}

// Defined for 1, 4, 7, etc.
val partialFunc2: PartialFunction[Int, String] = {
  case i if (i + 2) % 3 == 0 => "Something else"
}
printAndAssert("partialFunc2.isDefinedAt(1)", partialFunc2.isDefinedAt(1), true)
printAndAssert("partialFunc2.isDefinedAt(0)", partialFunc2.isDefinedAt(0), false)
println(s"partialFunc2(1) = ${partialFunc2(1)}")
try {
  println(partialFunc2(0))
} catch {
  case e: scala.MatchError => println("partialFunc2(0) = can't apply PartialFunctions where they are not defined")
}

val partialFunc3 = partialFunc1 orElse partialFunc2
printAndAssert("partialFunc3.isDefinedAt(0)", partialFunc3.isDefinedAt(0), false)
printAndAssert("partialFunc3.isDefinedAt(1)", partialFunc3.isDefinedAt(1), true)
printAndAssert("partialFunc3.isDefinedAt(2)", partialFunc3.isDefinedAt(2), true)
printAndAssert("partialFunc3.isDefinedAt(3)", partialFunc3.isDefinedAt(3), false)
println(s"partialFunc3(1) = ${partialFunc3(1)}")
println(s"partialFunc3(2) = ${partialFunc3(2)}")

---
# Type Safe Connections<a name="type-safe-connections"></a>

Chisel 可以检查许多连接的类型，包括：
* Bool/UInt 到 Clock

对于其他类型，Chisel 会让你连接它们，但可能会适当地截断/填充位。
* Bool/UInt 到 Bool/UInt
* Bundle 到 Bundle

In [ ]:
class Bundle1 extends Bundle {
  val a = UInt(8.W)
}

class Bundle2 extends Bundle1 {
  val b = UInt(16.W)
}

class BadTypeModule extends Module {
  val io = IO(new Bundle {
    val c  = Input(Clock())
    val in = Input(UInt(2.W))
    val out = Output(Bool())

    val bundleIn = Input(new Bundle2)
    val bundleOut = Output(new Bundle1)
  })
  
  //io.out := io.c // won't work due to different types

  // Okay, but Chisel will truncate the input width to 1 to match the output.
//   io.out := io.in

//   // Compiles; Chisel will connect the common subelements of the two Bundles (in this case, 'a').
//   io.bundleOut := io.bundleIn
}

println(getVerilog(new BadTypeModule))

---
# Type Generics（类型泛型）<a name="type-generics"></a>
Scala 的泛型类型（也称为多态）非常复杂，特别是在与继承结合使用时。

本节只是初步介绍；要了解更多，请查看[本教程](https://twitter.github.io/scala_school/type-basics.html)。

类可以在其类型上实现多态。一个很好的例子是序列，它们需要知道其包含的类型。

In [ ]:
val seq1 = Seq("1", "2", "3") // Type is Seq[String]
val seq2 = Seq(1, 2, 3)       // Type is Seq[Int]
val seq3 = Seq(1, "2", true)  // Type is Seq[Any]

有时，Scala 编译器需要帮助来确定多态类型，这需要用户显式地指定类型：

In [ ]:
//val default = Seq() // Error!
val default = Seq[String]() // User must tell compiler that default is of type Seq[String]
Seq(1, "2", true).foldLeft(default){ (strings, next) =>
    next match {
        case s: String => strings ++ Seq(s)
        case _ => strings
    }
}

函数也可以在其输入或输出类型上实现多态。以下示例定义了一个函数，用于计时运行一段代码所需的时间。它基于代码块的返回类型进行参数化。*请注意，`=> T` 语法表示没有参数列表的匿名函数，例如 `{ ... }` 与 `{ x => ... }`。*

译者注：
- 传名调用：block 是一个没有参数的匿名函数，只有在 block 被调用时才会执行其内容。这和传值调用（call-by-value）不同，传值调用会在函数被调用之前先求值所有参数。
- 类型参数化：block 的返回类型是泛型 T，这意味着 time 函数可以处理任何返回类型的代码块。可以认为，`block: => T`表示`block`的类型是` => T`。

In [ ]:
def time[T](block: => T): T = {
    val t0 = System.nanoTime()
    val result = block
    val t1 = System.nanoTime()
    val timeMillis = (t1 - t0) / 1000000.0
    println(s"Block took $timeMillis milliseconds!")
    result
}

// Adds 1 through a million
val int = time { (1 to 1000000).reduce(_ + _) }
println(s"Add 1 through a million is $int")

// Finds the largest number under a million that, in hex, contains "beef"
val string = time {
    (1 to 1000000).map(_.toHexString).filter(_.contains("beef")).last
}
println(s"The largest number under a million that has beef: $string")

## Chisel Type Hierarchy
要使用 Chisel 编写类型泛型代码，了解一些 Chisel 的类型层次结构是有帮助的。

`chisel3.Data` 是 Chisel 硬件类型的基类。
`UInt`、`SInt`、`Vec`、`Bundle` 等都是 `Data` 的实例。
`Data` 可以在 IOs 中使用，并支持 `:=`、wires、regs 等。

寄存器是 Chisel 中多态代码的一个好例子。
查看 `RegEnable`（带有 `Bool` 使能信号的寄存器）的实现[这里](https://github.com/freechipsproject/chisel3/blob/v3.0.0/src/main/scala/chisel3/util/Reg.scala#L10)。
apply 函数模板化为 `[T <: Data]`，这意味着 `RegEnable` 将适用于所有 Chisel 硬件类型。

某些操作仅定义在 `Bits` 的子类型上，例如 `+`。
这就是为什么你可以相加 `UInt` 或 `SInt`，但不能相加 `Bundle` 或 `Vec`。

<span style="color:blue">**示例：Type Generic ShiftRegister**<a name="type-generic-shift-register"></a></span><br>
在 Scala 中，对象和函数不是我们可以作为参数处理的唯一事物。
我们还可以将类型作为参数处理。

我们通常需要提供类型约束。
在这种情况下，我们希望能够将对象放入 bundle 中，连接（:=）它们，并用它们创建寄存器（RegNext）。
这些操作不能对任意对象执行；例如，wire := 3 是非法的，因为 3 是 Scala Int，不是 Chisel UInt。
如果我们使用类型约束来说类型 T 是 Data 的子类，那么我们可以对类型 T 的任何对象使用 :=，因为 := 对所有 Data 都定义。

这里是一个将类型作为参数的移位寄存器的实现。
*gen* 是一个类型为 T 的参数，告诉使用什么宽度，例如 `new ShiftRegister(UInt(4.W))` 是一个用于 4 位 UInt 的移位寄存器。
*gen* 还允许 Scala 编译器推断类型 T——如果你想更具体地写，可以写 `new ShiftRegister[UInt](UInt(4.W))`，但如果你省略 [UInt]，Scala 编译器足够聪明，可以自己找出答案。

In [ ]:
class ShiftRegisterIO[T <: Data](gen: T, n: Int) extends Bundle {
    require (n >= 0, "Shift register must have non-negative shift")
    
    val in = Input(gen)
    val out = Output(Vec(n + 1, gen)) // + 1 because in is included in out
    override def cloneType: this.type = (new ShiftRegisterIO(gen, n)).asInstanceOf[this.type]
}

class ShiftRegister[T <: Data](gen: T, n: Int) extends Module {
    val io = IO(new ShiftRegisterIO(gen, n))
    
    io.out.foldLeft(io.in) { case (in, out) =>
        out := in
        RegNext(in)
    }
}

visualize(() => new ShiftRegister(SInt(6.W), 3))
test(new ShiftRegister(SInt(6.W), 3)) { c => 
    println(s"Testing ShiftRegister of type ${c.io.in} and depth ${c.io.out.length}")
    for (i <- 0 until 10) {
        c.io.in.poke(i.S) // magic literal creation
        println(s"$i: ${c.io.out.indices.map { index => c.io.out(index).peek().litValue} }")
        c.clock.step(1)
    }}

我们通常建议避免将继承与类型泛型一起使用。
正确使用它可能非常棘手，并且很快会让人感到沮丧。

## Type Generics with Typeclasses（类型类）

上面的示例仅限于可以对任何 `Data` 实例执行的简单操作，例如 `:=` 或 `RegNext()`。
在生成 DSP 电路时，我们希望进行诸如加法和乘法的数学运算。
`dsptools` 库提供了编写类型参数化 DSP 生成器的工具。

这里是编写乘法累加模块的示例。
它可以用于生成 `FixedPoint`、`SInt` 甚至 `DspComplex[T]`（由 `dsptools` 提供的复数类型）的乘法累加（MAC）。
类型边界的语法有点不同，因为 `dsptools` 使用了类型类。
它们超出了此笔记本的范围。
请阅读 `dsptools` 的readme和文档，以获取有关使用类型类的更多信息。

`T <: Data : Ring` 意味着 `T` 是 `Data` 的子类型，同时也是一个 `Ring`。
`Ring` 在 `dsptools` 中定义为具有 `+` 和 `*` 运算（以及其他操作）的数字。

_`Ring` 的替代方案是 `Real`，但这不允许我们为 `DspComplex()` 创建 MAC，因为复数不是 `Real`。_

In [ ]:
import chisel3.experimental._
import dsptools.numbers._

class Mac[T <: Data : Ring](genIn : T, genOut: T) extends Module {
    val io = IO(new Bundle {
        val a = Input(genIn)
        val b = Input(genIn)
        val c = Input(genIn)
        val out = Output(genOut)
    })
    io.out := io.a * io.b + io.c
}

println(getVerilog(new Mac(UInt(4.W), UInt(6.W)) ))
println(getVerilog(new Mac(SInt(4.W), SInt(6.W)) ))
println(getVerilog(new Mac(FixedPoint(4.W, 3.BP), FixedPoint(6.W, 4.BP))))


<span style="color:red">**练习：Mac 作为对象**</span><br>

Mac `Module` 有少量输入和一个输出。
对于其他 Chisel 生成器来说，编写如下代码可能很方便：
```scala
val out = Mac(a, b, c)
```

在下面的 `Mac` 伴生对象中实现一个 `apply` 方法，该方法实现 `Mac` 功能。

In [ ]:
object Mac {
    def apply[T <: Data : Ring](a: T, b: T, c: T): T = {
        ??? // your code
    }
}

class MacTestModule extends Module {
    val io = IO(new Bundle {
        val uin = Input(UInt(4.W))
        val uout = Output(UInt())
        val sin = Input(SInt(4.W))
        val sout = Output(SInt())
        //val fin = Input(FixedPoint(16.W, 12.BP))
        //val fout = Output(FixedPoint())
    })
    // for each IO pair, do out = in * in + in
    io.uout := Mac(io.uin, io.uin, io.uin)
    io.sout := Mac(io.sin, io.sin, io.sin)
    //io.fout := Mac(io.fin, io.fin, io.fin)
}
println(getVerilog(new MacTestModule))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">

        a * b + c

</pre></article></div></section></div>

<span style="color:red">**练习：积分器**</span><br>
如图所示实现一个积分器。$n_1$ 是 `genReg` 的宽度，$n_2$ 是 `genIn` 的宽度。

不要忘记，`Reg`、`RegInit`、`RegNext`、`RegEnable` 等都是为类型 `T <: Data` 模板化的。

<img src="images/integrator.svg" alt="Integrator" style="width: 250px;"/>

In [ ]:
class Integrator[T <: Data : Ring](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn)
        val out = Output(genReg)
    })
    
    ??? // your code
}

test(new Integrator(SInt(4.W), SInt(8.W))) { c =>
    c.io.in.poke(3.S)
    c.io.out.expect(0.S)
    c.clock.step(1)
    c.io.in.poke(-4.S)
    c.io.out.expect(3.S)
    c.clock.step(1)
    c.io.in.poke(6.S)
    c.io.out.expect(-1.S)
    c.clock.step(1)
    c.io.out.expect(5.S)
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">

class Integrator\[T <: Data : Ring\](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn.cloneType)
        val out = Output(genReg.cloneType)
    })
    
    val reg = RegInit(genReg, Ring[T].zero) // init to zero
    reg := reg + io.in
    io.out := reg
}

</pre></article></div></section></div>

---
# Creating a Custom Type<a name="creating-a-custom-type"></a>

使 Chisel 强大的一个因素是它的可扩展性。
你可以添加自己的类型，这些类型具有适合你应用的操作和表示。
本节将介绍制作自定义类型的方法。

<span style="color:blue">**示例：DspComplex**</span><br>
`DspComplex` 是在 **dsptools** 中定义的自定义数据类型，定义在 [这里](https://github.com/ucb-bar/dsptools/blob/v1.0.0/src/main/scala/dsptools/numbers/chisel_concrete/DspComplex.scala#L59)。
关键的一行代码是：
```scala
class DspComplex[T <: Data:Ring](val real: T, val imag: T) extends Bundle { ... }
```

`DspComplex` 是一个类型泛型容器。
这意味着复数的实部和虚部可以是任何类型，只要它们满足类型约束 `T <: Data : Ring`。

`T <: Data` 意味着 `T` 是 `chisel3.Data` 的子类型，这是 Chisel 对象的基类型。
这意味着 `DspComplex` 仅适用于 Chisel 类型的对象，而不是任意的 Scala 类型。

`T : Ring` 意味着存在 `T` 的 Ring 类型类实现。
`Ring` 类型类定义了 `+` 和 `*` 运算符以及加法和乘法恒等元素（有关环的详细信息，请参阅[这篇维基百科文章](https://en.wikipedia.org/wiki/Ring_(mathematics))）。
**dsptools** 为常用的 Chisel 类型定义了类型类，定义在 [这里](https://github.com/ucb-bar/dsptools/tree/v1.0.0/src/main/scala/dsptools/numbers/chisel_types)。

**dsptools** 还为 `DspComplex` 定义了一个 `Ring` 类型类，因此我们可以使用复数重用我们的 MAC 生成器：

In [ ]:
println(getVerilog(new Mac(DspComplex(SInt(4.W), SInt(4.W)), DspComplex(SInt(6.W), SInt(6.W))) ))

<span style="color:red">**练习：符号-幅度数（Sign-magnitude Numbers）**</span><br>
假设你想使用符号-幅度表示法，并希望重用所有的 DSP 生成器。
类型类使这种临时多态成为可能。
以下示例给出了 SignMagnitude 类型的实现开端，以及一个 `Ring` 类型类的实现，该类型类将允许与 Mac 生成器一起使用。

填写 `+` 和 `*` 的实现。
你应该模仿 `unary_-()` 的实现。
下一块包含一个测试，用于检查使用 `SignMagnitude` 的 `Mac` 的正确性。

In [ ]:
class SignMagnitude(val magnitudeWidth: Option[Int] = None) extends Bundle {
    val sign = Bool()
    val magnitude = magnitudeWidth match {
        case Some(w) => UInt(w.W)
        case None    => UInt()
    }
    def +(that: SignMagnitude): SignMagnitude = {
        // Implement this!
    }
    def -(that: SignMagnitude): SignMagnitude = {
        this.+(-that)
    }
    def unary_-(): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := !this.sign
        result.magnitude := this.magnitude
        result
    }
    def *(that: SignMagnitude): SignMagnitude = {
        // Implement this!
    }
}
trait SignMagnitudeRing extends Ring[SignMagnitude] {
    def plus(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f + g
    }
    def times(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f * g
    }
    def one: SignMagnitude = {
        val one = Wire(new SignMagnitude(Some(1)))
        one.sign := false.B
        one.magnitude := 1.U
        one
    }
    def zero: SignMagnitude = {
        val zero = Wire(new SignMagnitude(Some(0)))
        zero.sign := false.B
        zero.magnitude := 0.U
        zero
    }
    def negate(f: SignMagnitude): SignMagnitude = {
        -f
    }
    
    // Leave unimplemented for this example
    def minusContext(f: SignMagnitude, g: SignMagnitude): SignMagnitude = ???
    def negateContext(f: SignMagnitude): SignMagnitude = ???
    def plusContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
    def timesContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
}
implicit object SignMagnitudeRingImpl extends SignMagnitudeRing

In [ ]:
import chisel3.experimental.BundleLiterals._

test(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))) { c =>
    c.io.a.poke(chiselTypeOf(c.io.a).Lit(_.sign -> false.B, _.magnitude -> 3.U))
    c.io.b.poke(chiselTypeOf(c.io.b).Lit(_.sign -> false.B, _.magnitude -> 3.U))
    c.io.c.poke(chiselTypeOf(c.io.c).Lit(_.sign -> false.B, _.magnitude -> 2.U))
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> false.B, _.magnitude -> 11.U))

    c.io.c.sign.poke(true.B)
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> false.B, _.magnitude -> 7.U))

    c.io.b.sign.poke(true.B)
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> true.B, _.magnitude -> 11.U))
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Look at the verilog to see if the output looks reasonable:

In [ ]:
println(getVerilog(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))))

`SignMagnitude` even works with `DspComplex`!

In [ ]:
println(getVerilog(new Mac(DspComplex(new SignMagnitude(Some(4)), new SignMagnitude(Some(4))), DspComplex(new SignMagnitude(Some(5)), new SignMagnitude(Some(5))))))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">
    // implementations for class SignMagnitude

    def +(that: SignMagnitude): SignMagnitude = {
      val result = Wire(new SignMagnitude())
      val signsTheSame = this.sign === that.sign
      when (signsTheSame) {
        result.sign      := this.sign
        result.magnitude := this.magnitude + that.magnitude
      } .otherwise {
        when (this.magnitude > that.magnitude) {
          result.sign      := this.sign
          result.magnitude := this.magnitude - that.magnitude
        } .otherwise {
          result.sign      := that.sign
          result.magnitude := that.magnitude - this.magnitude
        }   
      }   
      result
    }
    def *(that: SignMagnitude): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := this.sign ^ that.sign
        result.magnitude := this.magnitude * that.magnitude
        result
    }


</pre></article></div></section></div>